<a href="https://colab.research.google.com/github/vladgap/Various/blob/main/MLNN_func_approx_050422.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# data

In [100]:
data=[
[1127,	867,	850,	783,	1196,	1067,	962,	1543,	1430,	1544,	1440,	931,	770,	734,	655,	757,	2033,	1586,	1767,	1393,	1594,	1745,],
[1.313,	1.373,	1.389,	1.408,	1.293,	1.326,	1.355,	1.237,	1.25,	1.263,	1.253,	1.364,	1.406,	1.44,	1.46,	1.427,	1.207,	1.239,	1.22,	1.246,	1.25,	1.222,]
]
data=np.array(data).T
data=pd.DataFrame(data, columns=['sol','den']).sort_values(by=['sol'])
data.head()

,sol,den
14,655.0,1.460
13,734.0,1.440
15,757.0,1.427
12,770.0,1.406
3,783.0,1.408


In [101]:
pip install MLNN

# scaling

In [102]:
from sklearn import preprocessing
from MLNN import *

In [103]:
scaler_X = preprocessing.StandardScaler().fit(data[['sol']])
print(scaler_X.mean_, scaler_X.scale_)
X_sc = scaler_X.transform(data[['sol']]) # matrix
X_sc[0]

[1217.] [395.92572765]


array([-1.41945815])

In [104]:
scaler_y = preprocessing.StandardScaler().fit(data[['den']])
y_sc = scaler_y.transform(data[['den']])[:,0] # vector
y_sc

array([ 1.80801329,  1.55776923,  1.3951106 ,  1.13235434,  1.15737875,
        0.9196469 ,  0.71945165,  0.60684183,  0.49423201,  0.13137813,
       -0.03128051, -0.28152456, -0.86959809, -0.81954928, -0.78201267,
       -0.98220791, -0.65689064, -0.95718351, -0.81954928, -1.16989095,
       -1.19491536, -1.35757399])

# model fitting

In [115]:
layers=[1,2,1]

In [116]:
reg1=Regres(layers=layers)
reg1.set_hidden_activ('mrelu', param=0.01)
reg1.fit(np.array(X_sc), y_sc, epochs=1000, eta=0.01)
data1=go.Scatter(y=reg1.MSE_history[:], name='eta=0.01')

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation


In [117]:
reg2=Regres(layers=layers)
reg2.set_hidden_activ('mrelu', param=0.01)
reg2.fit(np.array(X_sc), y_sc, epochs=1000, eta=0.001)
data2=go.Scatter(y=reg2.MSE_history[:], name='eta=0.001')

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation


In [118]:
reg_fixed=Regres(layers=layers)
reg_fixed.set_hidden_activ('mrelu', param=0.01)
reg_fixed.set_weights(fixed_weights) # if no fixed_weights exist yet, go on
reg_fixed.fit(np.array(X_sc), y_sc, epochs=1000, eta=0)
data_fixed=go.Scatter(y=reg_fixed.MSE_history[:], name='fixed')

Model fitted.
self.weight_history - list of lists of weights propagation
self.weight_history_table - Pandas table of weights propagation
self.MSE_history - list of MSEs propagation


In [119]:
fig1=go.Figure()
fig1.add_trace(data1), print('MSE1=', reg1.MSE_history[-1])
fig1.add_trace(data2), print('MSE2=', reg2.MSE_history[-1])
fig1.add_trace(data_fixed), print('MSE_fixed=', reg_fixed.MSE_history[-1])
fig1.update_yaxes(range=[0, 1])
fig1.show()

MSE1= 4.066864233435758
MSE2= 0.0928021211578876
MSE_fixed= 0.04239543601178792


In [109]:
reg2.printWeights()


Layer 0 is the Input Layer
Layer 1 Neuron 0 : [0.88799513 1.26234191] mrelu
Layer 1 Neuron 1 : [-0.54150197 -0.77476279] mrelu
Layer 2 Neuron 0 : [-1.07094298 -0.8653522   1.35485947] linear



In [110]:
reg2.weight_history[-1]

[[[0.8879951279384605, 1.2623419103324058],
  [-0.5415019704839515, -0.7747627926207148]],
 [[-1.070942975851004, -0.8653522015534433, 1.3548594660492235]]]

In [111]:
fixed_weights=[[[0.48883429857980076, 1.0512199601618173],
  [-1.0506013145742896, 0.02763827850417239]],
 [[-0.9246202153963767, 1.0415687477051458, 0.46728254729148916]]]
fixed_MSE=0.042864610103826056

# predictions

In [112]:
chosen_model=reg_fixed

In [113]:
predics=scaler_y.inverse_transform([chosen_model.predict(np.array(X_sc))])[0]
errors=(predics-data.den)/data.den*100

In [114]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=data.sol, y=data.den, mode='markers', name='data'))
fig.add_trace(go.Scatter(x=data.sol, y=predics, mode='lines', name='prediction'))
fig.show()